In [1]:
import asyncio
import websockets
import json
import csv
import re
import datetime
import os

In [8]:
def pull_trades():
    
    msg = \
        {
            "jsonrpc": "2.0",
            "id": 1469,
            "method": "public/get_last_trades_by_currency_and_time",
            "params": {
                "currency": "BTC",
                "kind": "option",
                "start_timestamp": 0,
                "end_timestamp": 2566384979000,
                "count": 1000,
                "include_old": True,
                "sorting": "default"
            }
        }

    async def call_api(msg):
        async with websockets.connect('wss://test.deribit.com/ws/api/v2') as websocket:
            has_more = True
            step = 0
            timestamp = 0

            with open(r'trades.csv', 'w', newline='') as csvfile:
                fieldnames = ['trade_seq', 'trade_id', 'timestamp', 'tick_direction', 'price', 'instrument_name',
                              'index_price', 'direction', 'amount', 'iv', 'liquidation', 'strike', 'maturity', 'type']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

                while has_more:
                    msg['params']['start_timestamp'] = timestamp + 1
                    await websocket.send(json.dumps(msg))
                    # while websocket.open:
                    response = await websocket.recv()

                    # do something with the response...
                    data = json.loads(response)
                    trades = data['result']['trades']
                    has_more = data['result']['has_more']
                    timestamp = trades[-1]['timestamp']

                    # instrument parse
                    print(f"({step}) trades: {len(trades)}; timestamp: {timestamp}")

                    for trade in trades:
                        try:
                            del trade['block_trade_id']
                        except Exception:
                            pass

                        maturity, strike, xtype = re.findall("^BTC-(.+)-(.+)-(\w)$", trade['instrument_name'])[0]
                        trade['maturity'] = maturity
                        trade['strike'] = strike
                        trade['type'] = xtype
                        writer.writerow(trade)

                    step += 1

    asyncio.get_event_loop().run_until_complete(call_api(msg))


def pull_instruments():
    msg = \
        {
            "jsonrpc": "2.0",
            "id": 7617,
            "method": "public/get_instruments",
            "params": {
                "currency": "BTC",
                "kind": "option",
                "expired": True,
            }
        }

    async def call_api(msg):
        async with websockets.connect('wss://test.deribit.com/ws/api/v2', max_size=1_000_000_000) as websocket:
            with open(os.path.join(os.getcwd(), 'instruments.csv'), 'w', newline='') as csvfile:
                fieldnames = ["tick_size", "taker_commission", "strike", "settlement_period", "quote_currency",
                              "option_type", "min_trade_amount", "maker_commission", "kind", "is_active",
                              "instrument_name", "expiration_timestamp", "creation_timestamp", "contract_size",
                              "base_currency", ]
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

                await websocket.send(json.dumps(msg))
                response = await websocket.recv()

                # do something with the response...
                data = json.loads(response)
                instruments = data['result']

                writer.writerow(fieldnames)

                for instrument in instruments:
                    writer.writerow(instrument)

    asyncio.get_event_loop().run_until_complete(call_api(msg))


async def pull_book_summary():
    msg = \
        {
            "jsonrpc": "2.0",
            "id": 9344,
            "method": "public/get_book_summary_by_currency",
            "params": {
                "currency": "BTC",
                "kind": "option",
            }
        }

    async def call_api(msg):
        async with websockets.connect('wss://test.deribit.com/ws/api/v2', max_size=1_000_000_000) as websocket:
            date = datetime.datetime.now().strftime("%Y%m%d")
            filename = os.path.join(os.getcwd(), "_input", "quotes", f"{date}.csv")

            with open(filename, 'w', newline='') as csvfile:
                fieldnames = ["volume", "underlying_price", "underlying_index", "quote_currency", "open_interest",
                              "mid_price", "mark_price", "low", "last", "interest_rate", "instrument_name", "high",
                              "estimated_delivery_price", "creation_timestamp", "bid_price", "base_currency",
                              "ask_price", "maturity", "strike", "type"]
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

                await websocket.send(json.dumps(msg))
                response = await websocket.recv()

                # do something with the response...
                data = json.loads(response)
                instruments = data['result']

                writer.writerow(dict(zip(fieldnames, fieldnames)))

                for instrument in instruments:
                    maturity, strike, xtype = re.findall("^BTC-(.+)-(.+)-(\w)$", instrument['instrument_name'])[0]
                    instrument['maturity'] = maturity
                    instrument['strike'] = strike
                    instrument['type'] = xtype

                    try:
                        del instrument['price_change']
                    except:
                        pass

                    writer.writerow(instrument)

    await call_api(msg)


if __name__ == '__main__':
    await pull_book_summary()